In [26]:
import re
from tkinter import filedialog 
from collections import Counter, defaultdict

In [2]:
f_path = filedialog.askopenfile(title="Select text file.").name
f = open(f_path)

In [27]:
full_text = f.read(500000)

In [28]:
def normalize(text):
    text = text.lower()

    text = re.sub('[^a-z0-9]+', ' ', text)

    text=  re.sub('/s+', ' ', text)

    return text.strip()

In [29]:
texts = full_text.split()

clean_texts = [text for text in texts if text != '[INST]' and text!= '[/INST]']

words = []
for text in clean_texts:
    text = normalize(text)
    words.extend(text.split())


word_freq = Counter(words)


In [30]:
bpe_vocab = {}
for word, freq in word_freq.items():
    chars = list(word)
    chars.append('</w>')
    bpe_vocab[tuple(chars)] = freq
    # bpe_vocab[freq] = tuple(chars)

In [31]:
def get_stats(vocab):
    pairs = defaultdict(int)

    # for freq, word in vocab.items():
    for word, freq in vocab.items():
        symbols = word
        for i in range(len(symbols)-1):
            # ****
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

In [32]:
pairs = get_stats(bpe_vocab)
best_pair = max(pairs, key=pairs.get)


In [33]:
# ******

def merge_vocab(pair, vocab):
    new_vocab = {}
    bigram = ''.join(pair)

    for word, freq in vocab.items():
        w = list(word)

        i = 0
        new_word = []

        while i < len(w):
            if i < len(w)-1 and (w[i], w[i+1]) == pair:
                new_word.append(bigram)
                i+=2
            else:
                new_word.append(w[i])
                i+=1

        new_vocab[tuple(new_word)] = freq

    return new_vocab

In [34]:
bpe_vocab = merge_vocab(best_pair, bpe_vocab)

In [35]:
bpe_vocab

{('o', 'n', 'n', 'e', 'c', 't', 'i', 'o', 'n', '</w>'): 1,
 ('h', 'e', 'a', 'l', 't', 'h', 'y', '</w>'): 41,
 ('a', 'n', 'd', '</w>'): 2277,
 ('h', 'a', 'p', 'p', 'y', '</w>'): 217,
 ('h', 'e', 'l', 'l', 'o', '</w>'): 228,
 ('i', '</w>'): 2625,
 ('a', 'm', '</w>'): 146,
 ('r', 'e', 'a', 'l', 'l', 'y', '</w>'): 303,
 ('u', 'p', 's', 'e', 't', '</w>'): 28,
 ('a', 'b', 'o', 'u', 't', '</w>'): 776,
 ('t', 'h', 'e</w>'): 1927,
 ('v', 'e', 'r', 'y', '</w>'): 221,
 ('l', 'o', 'n', 'g', '</w>'): 54,
 ('w', 'a', 'i', 't', '</w>'): 14,
 ('a', 't', '</w>'): 168,
 ('s', 't', 'o', 'r', 'e</w>'): 13,
 ('t', 'o', 'd', 'a', 'y', '</w>'): 357,
 ('u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd', '</w>'): 241,
 ('t', 'h', 'a', 't', '</w>'): 1331,
 ('w', 'a', 'i', 't', 'i', 'n', 'g', '</w>'): 20,
 ('c', 'a', 'n', '</w>'): 1797,
 ('b', 'e</w>'): 628,
 ('f', 'r', 'u', 's', 't', 'r', 'a', 't', 'i', 'n', 'g', '</w>'): 82,
 ('i', 't', '</w>'): 2335,
 ('m', 'a', 'k', 'e</w>'): 383,
 ('a', 'n', 'y', 'o', 'n', 'e